In [1]:
import psycopg2

In [3]:
from sqlalchemy import create_engine, Table, Column, Integer,Float, String, MetaData 
import pandas as pd

UPLOADING TO POSTGRES AND CONNECTION

In [4]:
df = pd.read_csv('data/AB_NYC_2019.csv')

In [36]:
#Connecting our postgres database to our IDE
connection = psycopg2.connect(user = "postgres",
                              password = "nagar7754",
                              database = "airbnb-ny",
                              port = "5432")

In [37]:
connection

<connection object at 0x1179ebb50; dsn: 'user=postgres password=xxx dbname=airbnb-ny port=5432', closed: 0>

In [25]:
cursor=connection.cursor() #Cursor to execute commands and fetch data.
cursor

<cursor object at 0x124ce8d60; closed: 0>

In [41]:
from sqlalchemy import create_engine

db_name = 'airbnb-ny'
db_user = 'postgres'  # replace with your PostgreSQL username
db_password = 'nagar7754'  # replace with your PostgreSQL password
db_host = 'localhost'  # or the host of your PostgreSQL server
db_port = '5432'  # default PostgreSQL port

#Create a connection string
connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

engine = create_engine(connection_string)

# Read the CSV file into a DataFrame
csv_file_path = 'data/AB_NYC_2019.csv'  # replace with your CSV file path
df = pd.read_csv(csv_file_path)

table_name = 'airbnb_listings'  #Uploading dataframe to sql directly
df.to_sql(table_name, engine, if_exists='replace', index=False)

print("Data uploaded successfully!")

Data uploaded successfully!


EXTRACTING DATA BACK FROM POSTGRES USING QUERY

In [55]:
# Now getting the data back using SQL alchemy and putting it in a df
query = " SELECT * FROM airbnb_listings"

df = pd.read_sql(query,engine)
df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,None,NaN,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,None,NaN,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,None,NaN,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,None,NaN,6,2


DATA TRANSFORMATION (NORMALIZE DATA)

In [56]:
missing_values = df.isnull().sum()
print(missing_values)

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64


In [62]:
# SO there is last_review and reviews_per_month which have 10052 missing values, so we will drop them.                      
# and for the name and host name which have 16 and 21 missing value,s rather than dropping them we will add Absent to them.
df_cleanedd = df['name'].fillna('Absent', inplace = True)
df_cleanedd = df['host_name'].fillna('Absent', inplace =  True)
df_cleanedd = df.drop(['last_review','reviews_per_month'], axis =1)

In [63]:
df_cleanedd

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48890,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,2,9
48891,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,2,36
48892,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,1,27
48893,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,6,2


In [66]:
missing_values = df_cleanedd.isnull().sum()
print(missing_values)
#Now no missing value.

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64


CALCULATE METRICS

In [67]:
avg_price_per_neighborhood_group = df_cleanedd.groupby('neighbourhood_group')['price'].mean()

# Display the results
print("Average price of rooms based on neighborhood group:")
print(avg_price_per_neighborhood_group)

Average price of rooms based on neighborhood group:
neighbourhood_group
Bronx             87.496792
Brooklyn         124.383207
Manhattan        196.875814
Queens            99.517649
Staten Island    114.812332
Name: price, dtype: float64


In [68]:
count_per_neighborhood_group = df_cleanedd.groupby('neighbourhood_group')['id'].count()

# Display the results
print("Number of listings per neighborhood group:")
print(count_per_neighborhood_group)

Number of listings per neighborhood group:
neighbourhood_group
Bronx             1091
Brooklyn         20104
Manhattan        21661
Queens            5666
Staten Island      373
Name: id, dtype: int64


In [69]:
avg_minimum_nights_per_neighborhood_group = df.groupby('neighbourhood_group')['minimum_nights'].mean()

# Display the results
print("Average minimum nights per neighborhood group:")
print(avg_minimum_nights_per_neighborhood_group)

Average minimum nights per neighborhood group:
neighbourhood_group
Bronx            4.560953
Brooklyn         6.056556
Manhattan        8.579151
Queens           5.181433
Staten Island    4.831099
Name: minimum_nights, dtype: float64


In [70]:
#Adding the useful metrics directly to the dataframe.
df_cleanedd['avg_price_neighborhood'] = df.groupby('neighbourhood')['price'].transform('mean')

# Calculate the total number of listings per neighborhood
df_cleanedd['listings_count_neighborhood'] = df.groupby('neighbourhood')['id'].transform('count')


In [ ]:
df_cleanedd

In [73]:
# Define the name of the new table
new_table_name = 'airbnb_listings_enhanced'

# Upload the modified DataFrame back to PostgreSQL
df_cleanedd.to_sql(new_table_name, engine, if_exists='replace', index=False)
# Now this whole ETL workflow is done.

895